In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DL_project/Song_emotion_compare/cat_proba_lyrics.csv')
df.mean()[:5]

<ipython-input-89-c583033134da>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  list(df.mean()[:5])


[0.08725205660623461,
 0.15137762897081344,
 0.14268008641334992,
 0.2445885912322183,
 0.3741016367773835]

#코드 제작

In [ ]:
def cos_recommend(proba):
    ran_val = 30
    # 우리가 만들어둔 노래 가사들을 감정분석 돌린 결과 csv파일을 읽어온다.
    df = pd.read_csv(os.path.join(filePath, 'data', 'cat_proba_lyrics.csv'))
    # st.dataframe(df)
    
    # 입력 예시 (사용자와의 챗봇 대화를 통해서 상대방의 감정을 얻어낸 값을 user_emotion에에 넣으면 됨.)
    user_emotion = torch.tensor(proba) # 기분 예시
    
    # 우리의 노래 가사 분석 데이터를 tensor type으로 변환
    lyric_emotion = torch.from_numpy(df.values[:,3:8].astype(float))
    
    #코사인 유사도 분석 함수를 적용
    df['similarity'] = pd.Series(F.cosine_similarity(user_emotion,lyric_emotion))
    
    # 기분전환 노래를 위한 감정 변환환 값
    revese_emotion = torch.tensor(reverse(user_emotion))
    
    # 노래와 유저기분의 차이값 저장
    df['diff'] = pd.Series(F.cosine_similarity(revese_emotion, lyric_emotion))
    random_add = np.random.rand(len(df['similarity']))/ran_val

    df['similarity']  += random_add
    random_add = np.random.rand(len(df['diff']))/ran_val
    df['diff']  += random_add
    
    #유사도 높은 것과 낮은 것 출력
    similar = df.iloc[np.argmax(df['similarity'])]
    different = df.iloc[np.argmin(df['diff'])]
    return similar['Songs_title'], different['Songs_title'], similar['link'], different['link']

In [ ]:
def reverse(user_emotion):
    emotion_div = 1.5
    reverse_emotion = [0,0,0,0,0]
    
    if np.argmax(user_emotion) == 0:
        user_emotion[0] /= 3
        total = sum(user_emotion)
        for i in range(0,5):
            reverse_emotion[i] = user_emotion[i] / total
        
    else:
        for i in range(1,5):
            # user emotion 리스트를 특정수로 나누기
            add_num = user_emotion[i] / emotion_div
            
            # 나눈 값을 기쁨에 덯개주기
            reverse_emotion[0] += add_num
            
            # 바뀐
            reverse_emotion[i] = user_emotion[i] - add_num
            
        reverse_emotion[0] += user_emotion[0]
    return reverse_emotion
print([0.01,0.9,0.03,0.03,0.03])
print(reverse([0.01,0.9,0.03,0.03,0.03]))
print(sum(reverse([0.01,0.9,0.03,0.03,0.03])))

In [ ]:
cos_recommend()

tensor([0.0745, 0.1028, 0.0854, 0.3892, 0.3482], dtype=torch.float64)
유사도 높은 :  Off the menu - Sunshine
반대 유사도 높은 :  김진표 - 오롤롤롤로
